# 01 — Explore ShapeNetCore

Use this notebook with the **Python (pytorch3d)** kernel.

In [ ]:
# If the kernel isn't set yet: Kernel → Change Kernel → Python (pytorch3d)
import sys, os, platform, torch
print("Python:", sys.version)
print("PyTorch:", torch.__version__, " MPS avail:", torch.backends.mps.is_available())




Python: 3.10.18 (main, Jun  5 2025, 08:37:47) [Clang 14.0.6 ]
PyTorch: 2.8.0  MPS avail: True


## Paths
Set this to your local dataset path. For this project it's expected at `./shapeNetCore`. Adjust if yours differs.

In [ ]:
from pathlib import Path
SHAPENET_PATH = Path('../data/shapeNetCore').expanduser().resolve()
assert SHAPENET_PATH.exists(), f"ShapeNetCore folder not found at {SHAPENET_PATH}"
print("Using SHAPENET_PATH:", SHAPENET_PATH)



Using SHAPENET_PATH: /Users/brageramberg/Desktop/3DCNN/data/shapeNetCore


## Imports

In [ ]:
import sys, os, importlib.util, pathlib
print("Python:", sys.executable)
print("First few sys.path entries:", sys.path[:5])

# Is there a local matplotlib module/folder?
candidates = []
for base in [pathlib.Path('.'), pathlib.Path('..')]:
    for name in ["matplotlib.py", "matplotlib"]:
        p = base / name
        if p.exists():
            candidates.append(p.resolve())
print("Local candidates:", candidates)

# Where would Python import matplotlib from?
spec = importlib.util.find_spec("matplotlib")
print("Import spec for matplotlib:", spec)



Python: /Users/brageramberg/opt/miniconda3/envs/pytorch3d/bin/python
First few sys.path entries: ['/Users/brageramberg/opt/miniconda3/envs/pytorch3d/lib/python310.zip', '/Users/brageramberg/opt/miniconda3/envs/pytorch3d/lib/python3.10', '/Users/brageramberg/opt/miniconda3/envs/pytorch3d/lib/python3.10/lib-dynload', '', '/Users/brageramberg/opt/miniconda3/envs/pytorch3d/lib/python3.10/site-packages']
Local candidates: []
Import spec for matplotlib: ModuleSpec(name='matplotlib', loader=<_frozen_importlib_external.SourceFileLoader object at 0x120551540>, origin='/Users/brageramberg/opt/miniconda3/envs/pytorch3d/lib/python3.10/site-packages/matplotlib/__init__.py', submodule_search_locations=['/Users/brageramberg/opt/miniconda3/envs/pytorch3d/lib/python3.10/site-packages/matplotlib'])


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter, defaultdict

from pytorch3d.datasets import ShapeNetCore, collate_batched_meshes
from pytorch3d.renderer import (
    OpenGLPerspectiveCameras, PointLights, RasterizationSettings, look_at_view_transform
)
from pytorch3d.structures import Meshes
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Device:", device)



Device: mps


## Load dataset and Category overview

In [ ]:
from pytorch3d.datasets import ShapeNetCore
import os
from pathlib import Path

synset = SHAPENET_PATH / "03211117"
print("Synset exists:", synset.exists())
print("Contents of synset:", os.listdir(synset)[:5])

shapenet_dataset = ShapeNetCore(str(SHAPENET_PATH), version=2, load_textures=False)
print("Total models:", len(shapenet_dataset))



Synset exists: True
Contents of synset: ['91db88d4d0956e92a9a12c50dc330a66', 'eb2b450565c77e3efc7ff8f848d1c6c9', '871dcd6740ed8d2891a7ac029a825f73', 'aa73f1b28817376c6cf19e722198d69', '4516c31dae48a0efbff4c0ca0d98b77']


/Users/brageramberg/opt/miniconda3/envs/pytorch3d/lib/python3.10/site-packages/pytorch3d/datasets/shapenet/shapenet_core.py:116: UserWarning: The following categories are included in ShapeNetCore ver.2's official mapping but not found in the dataset location /Users/brageramberg/Desktop/3DCNN/data/shapeNetCore: 04256520, 03710193, 03761084, 03759954, 04099429, 03337140, 03948459, 02933112, 02801938, 02871439, 04330267, 04401088, 03790512, 03467517, 04468005, 02924116, 04530566, 02876657, 03593526, 02747177, 02691156, 02954340, 03928116, 02818832, 04225987, 04004475, 04379243, 03938244, 02942699, 03325088, 03691459, 04074963, 03991062, 04460130, 03797390, 04090263, 03207941, 02946921, 02828884, 03001627, 04554684, 03513137, 02773838, 02958343, 03636649, 03624134, 02843684, 02880940, 03261776, 03085013
  warnings.warn(msg)


Total models: 3891


In [ ]:
#look at one sample

sample = shapenet_dataset[0]
print("Keys:", sample.keys())
print("Synset:", sample["synset_id"])
print("Model ID:", sample["model_id"])
print("Verts:", sample["verts"].shape, "Faces:", sample["faces"].shape)



Keys: dict_keys(['synset_id', 'model_id', 'verts', 'faces', 'textures', 'label'])
Synset: 02808440
Model ID: cb38405e384c79c05e4e385b035691bb
Verts: torch.Size([6994, 3]) Faces: torch.Size([28700, 3])


In [8]:
from collections import Counter

cat_counts = Counter()
for i in range(len(shapenet_dataset)):
    try:
        cat_counts[shapenet_dataset[i]["synset_id"]] += 1
    except Exception as e:
        # skip broken model
        print("Some model is broken")
        continue

print("Categories found:", len(cat_counts))
print("Example:", list(cat_counts.items())[:10])

Categories found: 5
Example: [('02808440', 856), ('03642806', 460), ('03046257', 651), ('03211117', 1093), ('02992529', 831)]
